In [15]:
import pandas as pd
import numpy as np
import joblib
import ast
import os
from typing import List

# =====================================================
# 1. CONFIGURATION
# =====================================================
DATA_PATH = r"C:\DineSense AI\data\processed\clean_restaurants.csv"
MODEL_DIR = r"C:\DineSense AI\models"
MODEL_PATH = os.path.join(MODEL_DIR, "rating_prediction_model.joblib")

REQUIRED_COLUMNS = [
    'restaurant_name', 'city', 'locality',
    'average_cost_for_two', 'votes', 'price_range',
    'cuisines', 'aggregate_rating'
]

FEATURE_COLUMNS = [
    'average_cost_for_two',
    'votes',
    'price_range',
    'cuisines',
    'city',
    'locality'
]

# =====================================================
# 2. UTILITY FUNCTIONS
# =====================================================
def normalize_cuisines(value) -> str:
    """Converts cuisines into clean, comma-separated string."""
    if pd.isna(value):
        return "unknown"
    if isinstance(value, list):
        return ", ".join(map(str, value))
    if isinstance(value, str):
        try:
            parsed = ast.literal_eval(value)
            if isinstance(parsed, list):
                return ", ".join(map(str, parsed))
        except Exception:
            pass
        return value.replace("[", "").replace("]", "").replace("'", "")
    return str(value)

def validate_dataset(df: pd.DataFrame):
    """Ensure dataset has all required columns."""
    missing = [c for c in REQUIRED_COLUMNS if c not in df.columns]
    if missing:
        raise ValueError(f"❌ Dataset missing required columns: {missing}")

# =====================================================
# 3. LOAD DATA
# =====================================================
df = pd.read_csv(DATA_PATH)
validate_dataset(df)
print(f"✅ Dataset loaded successfully: {df.shape}")

df = df.copy()
df["cuisines"] = df["cuisines"].apply(normalize_cuisines)

# =====================================================
# 4. TRAIN & SAVE MODEL (if not already saved)
# =====================================================
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor

if not os.path.exists(MODEL_PATH):
    # Split features & target
    X = df[FEATURE_COLUMNS].copy()
    y = df['aggregate_rating']

    # Convert text features
    for col in ["cuisines", "city", "locality"]:
        X[col] = X[col].astype(str)

    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # Preprocessor
    cat_features = ['cuisines', 'city', 'locality']
    num_features = ['average_cost_for_two', 'votes', 'price_range']

    preprocessor = ColumnTransformer([
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_features),
        ('num', 'passthrough', num_features)
    ])

    # Pipeline
    model = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', RandomForestRegressor(n_estimators=200, max_depth=15, random_state=42, n_jobs=-1))
    ])

    # Train model
    model.fit(X_train, y_train)
    print("✅ Model trained successfully")

    # Save model
    os.makedirs(MODEL_DIR, exist_ok=True)
    joblib.dump(model, MODEL_PATH)
    print(f"✅ Trained model saved at: {MODEL_PATH}")
else:
    # Load pre-trained model
    model = joblib.load(MODEL_PATH)
    print("✅ Rating prediction model loaded")

# =====================================================
# 5. PREDICT RATINGS
# =====================================================
X = df[FEATURE_COLUMNS].copy()
for col in ["cuisines", "city", "locality"]:
    X[col] = X[col].astype(str)

df["predicted_rating"] = model.predict(X)
print("✅ Predicted ratings added")

# =====================================================
# 6. FINAL SCORING LOGIC (PRODUCTION)
# =====================================================
df["final_score"] = (
    0.6 * df["predicted_rating"] +
    0.3 * df["aggregate_rating"] +
    0.1 * np.log1p(df["votes"])
)

# =====================================================
# 7. GLOBAL RECOMMENDATIONS
# =====================================================
def get_top_global_restaurants(top_n: int = 10) -> pd.DataFrame:
    return (
        df.sort_values("final_score", ascending=False)
          .head(top_n)
          .loc[:, [
              "restaurant_name",
              "city",
              "cuisines",
              "average_cost_for_two",
              "predicted_rating",
              "aggregate_rating",
              "votes"
          ]]
          .reset_index(drop=True)
    )

print("\n🍽️ Top Restaurant Recommendations (ALL CITIES):\n")
print(get_top_global_restaurants())

# =====================================================
# 8. CITY-WISE RECOMMENDATIONS
# =====================================================
def recommend_by_city(city_name: str, top_n: int = 5) -> pd.DataFrame:
    city_df = df[df["city"].str.lower() == city_name.lower()]
    if city_df.empty:
        return pd.DataFrame(columns=[
            "restaurant_name", "city", "cuisines",
            "average_cost_for_two", "predicted_rating",
            "aggregate_rating", "votes"
        ])

    return (
        city_df.sort_values("final_score", ascending=False)
               .head(top_n)
               .loc[:, [
                   "restaurant_name",
                   "city",
                   "cuisines",
                   "average_cost_for_two",
                   "predicted_rating",
                   "aggregate_rating",
                   "votes"
               ]]
               .reset_index(drop=True)
    )



✅ Dataset loaded successfully: (7299, 17)
✅ Rating prediction model loaded
✅ Predicted ratings added

🍽️ Top Restaurant Recommendations (ALL CITIES):

                       restaurant_name       city  \
0                                 Toit  Bangalore   
1            AB's - Absolute Barbecues  Hyderabad   
2                      Yellow Dog Eats    Orlando   
3              AB's Absolute Barbecues      Dubai   
4  Pom Pom's Teahouse and Sandwicheria    Orlando   
5       Hollerbach's Willow Tree Caf��    Orlando   
6                      Barbeque Nation    Kolkata   
7                             Truffles  Bangalore   
8                     Earl of Sandwich    Orlando   
9                             Yauatcha     London   

                                        cuisines  average_cost_for_two  \
0               ['italian', 'american', 'pizza']                  2000   
1  ['european', 'mediterranean', 'north indian']                  1500   
2                ['american', 'bbq', 'sandw